## Random Walk Model

The code implementation of a **random walk model in a square lattice (2D)**.

- The walker starts at the origin (0,0) and moves randomly in the lattice.
- The walker can move in four directions: up, down, left, right. *All with equal probability*.
- The walker can move only one step at a time.

